### Connect to workspace

In [49]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Found the config file in: /home/demouser/repos/AML/aml_config/config.json
jkamllab
jkamllab
eastus2
952a710c-8d9c-40c1-9fec-f752138cc0b3


### Retrieve the model

In [50]:
from azureml.core import Experiment

exp = Experiment(ws, name='aerial-classifier-train')
tags = {"RunName": "Pretrain2"}
runs = exp.get_runs(tags=tags)
try:
    run = next(runs)
except:
    print("Could not find the run with {}".format(tags))
    
run.download_file('outputs/aerial_classifier_weights.hd5', 'aerial_classifier_weights.hd5')

In [51]:
%%sh

ls -l

total 6780
-rw-rw-r-- 1 demouser demouser 3210976 Oct 23 04:58 aerial_classifier.hd5
-rw-rw-r-- 1 demouser demouser 1075904 Oct 23 05:33 aerial_classifier_weights.hd5
-rw-rw-r-- 1 demouser demouser    6787 Oct 22 17:53 dataprep.ipynb
-rw-rw-r-- 1 demouser demouser    4964 Oct 22 17:53 extract.py
-rw-rw-r-- 1 demouser demouser   51620 Oct 23 05:01 fine-tune.ipynb
-rw-rw-r-- 1 demouser demouser    4778 Oct 22 17:53 fine-tune.py
-rw-rw-r-- 1 demouser demouser    2736 Oct 22 17:53 gen_aerial_tfrecords.py
-rw-rw-r-- 1 demouser demouser  740190 Oct 22 17:53 model1.png
-rw-rw-r-- 1 demouser demouser  742617 Oct 22 17:53 model2.png
-rw-rw-r-- 1 demouser demouser    6880 Oct 22 17:53 model.png
-rw-rw-r-- 1 demouser demouser 1055525 Oct 22 22:02 sanbox.ipynb
-rw-rw-r-- 1 demouser demouser    9799 Oct 22 17:53 TrainingCodeSnippets.ipynb
-rw-rw-r-- 1 demouser demouser    5937 Oct 22 17:53 train.py


### Assemble the model
Stack the *top_model* on top of **VGG16** "trunk"

In [54]:
import tensorflow.keras as keras
from tensorflow.keras.applications import vgg16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input
from tensorflow.keras.regularizers import l1_l2

# Load the top model
top_model = keras.models.load_model('aerial_classifier.hd5')
top_model.summary()


# Create a featurizer
trunk = vgg16.VGG16(
                weights = 'imagenet', 
                input_shape=(224,224,3), 
                include_top = False,
                pooling = 'avg')


# Define network
def fcn_classifier(input_shape=(2048,), units=512, classes=6,  l1=0.01, l2=0.01):
    features = Input(shape=input_shape)
    x = Dense(units, activation='relu')(features)
    x = Dropout(0.5)(x)
    y = Dense(classes, activation='softmax', kernel_regularizer=l1_l2(l1=l1, l2=l2))(x)
    model = Model(inputs=features, outputs=y)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

top_model = fcn_classifier(input_shape=(512,), units=512, l1=0.005, l2=0.005)
top_model.load_weights('aerial_classifier_weights.hd5')

full_model = Model(inputs=trunk.input, outputs=top_model(trunk.output))

# Set the first 15 layers (up to the last conv block)
# to non-trainable 
for layer in trunk.layers[:15]:
    layer.trainable = False
    
for layer in trunk.layers[15:]:
    layer.trainable = True
    
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate
optimizer = SGD(lr=0.00001, momentum=0.9)
loss = 'categorical_crossentropy'
metrics = ['accuracy']
full_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
full_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 265,734
Trainable params: 265,734
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
bloc

In [55]:
trunk.layers[15:]

### Create a dataset

In [56]:
import tensorflow as tf
import numpy as np

# Returns a dataset based on a list of TFRecords files passsed as a parameters. 
def create_dataset(files, batch_size=32, prefetch_buffer_size=1, train=True, buffer_size=10000, num_parallel_calls=4):
    IMAGE_SHAPE = (224, 224, 3,)
    NUM_CLASSES = 6
          
    # Extract image and label from proto Example
    def _parse(example_proto):
        features = {'label': tf.FixedLenFeature((), tf.int64, default_value=0),
                    'image': tf.FixedLenFeature((), tf.string, default_value="")}
        parsed_features = tf.parse_single_example(example_proto, features)
        label = parsed_features['label']
        label = tf.one_hot(label, NUM_CLASSES)
        image = image = tf.decode_raw(parsed_features['image'], tf.uint8)
        image = tf.cast(image, tf.float32)
        image = tf.reshape(image, IMAGE_SHAPE)
                                                                  
        # Pre-process image data for VGG16
        #   RGB -> BGR
        image = image[..., ::-1]
        #   Substract the Imagenet mean for each channel
        imagenet_mean=tf.constant(-np.array([103.939, 116.779, 123.68]), dtype=tf.float32)
        image = tf.nn.bias_add(image, imagenet_mean)
        #image = tf.subtract(image, imagenet_mean)
        #image = resnet50.preprocess_input(image)
        
        return image, label

    dataset = tf.data.TFRecordDataset(files)
    dataset = dataset.map(_parse, num_parallel_calls=num_parallel_calls)
    #dataset = dataset.map(_parse)
    if train:
        dataset = dataset.shuffle(buffer_size)
    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.prefetch(buffer_size=prefetch_buffer_size)
    dataset = dataset.repeat()

    return dataset

In [57]:
def get_num_of_records(files):
    count = 0
    for file in files:
        for record in tf.python_io.tf_record_iterator(file):
            count += 1
    return count

In [58]:
%%sh

ls -l ../datasets/tfrecords/aerialmed/training

total 2598956
-rw-rw-r-- 1 demouser demouser 2661324051 Oct 22 17:53 aerialmed_train.tfrecords


In [59]:
import os

training_dir = '../datasets/tfrecords/aerialmed/training'
validation_dir = '../datasets/tfrecords/aerialmed/validation'

training_files = [os.path.join(training_dir, file) for file in os.listdir(training_dir)]
validation_files = [os.path.join(validation_dir, file) for file in os.listdir(validation_dir)]
                                                                          
train_dataset = create_dataset(training_files, batch_size = 32, train=True)
valid_dataset = create_dataset(validation_files, batch_size = 32, train=False)
                                           

In [60]:
batch_size = 32

steps_per_epoch = get_num_of_records(training_files)//batch_size
validation_steps = get_num_of_records(validation_files)//batch_size

    #print("Starting training:")
    #print("    Epochs: ", FLAGS.epochs)
    #print("    Steps per epoch: ", steps_per_epoch)
    #print("    Validation steps: ", validation_steps)
    #print("    Dropout prob: ", FLAGS.dropout_prob)
    #print("    L1: ", FLAGS.l1)
    #print("    L2: ", FLAGS.l2)

epochs = 50

full_model.fit(train_dataset,
        epochs = epochs,
        steps_per_epoch = steps_per_epoch,
        validation_data = valid_dataset,
        validation_steps = validation_steps)

Epoch 1/50
552/552 [==============================] - 42s 76ms/step - loss: 0.1631 - acc: 0.9610 - val_loss: 0.1915 - val_acc: 0.9508
Epoch 2/50
552/552 [==============================] - 40s 73ms/step - loss: 0.1554 - acc: 0.9613 - val_loss: 0.1851 - val_acc: 0.9505
Epoch 3/50
552/552 [==============================] - 40s 73ms/step - loss: 0.1482 - acc: 0.9636 - val_loss: 0.1835 - val_acc: 0.9508
Epoch 4/50
552/552 [==============================] - 40s 73ms/step - loss: 0.1424 - acc: 0.9626 - val_loss: 0.1795 - val_acc: 0.9521
Epoch 5/50
552/552 [==============================] - 40s 73ms/step - loss: 0.1355 - acc: 0.9652 - val_loss: 0.1742 - val_acc: 0.9512
Epoch 6/50
552/552 [==============================] - 40s 73ms/step - loss: 0.1340 - acc: 0.9669 - val_loss: 0.1705 - val_acc: 0.9495
Epoch 7/50
357/552 [==================>...........] - ETA: 13s - loss: 0.1229 - acc: 0.9683

KeyboardInterrupt: 

In [43]:
full_model.get_config()

{'name': 'model_9',
 'layers': [{'name': 'input_10',
   'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 224, 224, 3),
    'dtype': 'float32',
    'sparse': False,
    'name': 'input_10'},
   'inbound_nodes': []},
  {'name': 'block1_conv1',
   'class_name': 'Conv2D',
   'config': {'name': 'block1_conv1',
    'trainable': False,
    'dtype': 'float32',
    'filters': 64,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'same',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None,
      'dtype': 'float32'}},
    'bias_initializer': {'class_name': 'Zeros',
     'config': {'dtype': 'float32'}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_c